In [1]:
import math
from collections import defaultdict
emit = defaultdict(int)
transit = defaultdict(int)
context = defaultdict(int)

In [2]:
# # テスト
# input_file_train = open("../test/05-train-input.txt","r")
# input_file = open("../test/05-test-input.txt","r")

# 本番
input_file_train = open("../data/wiki-en-train.norm_pos","r")
input_file = open("../data/wiki-en-test.norm","r")

inp_train = input_file_train.readlines()
inp = input_file.readlines()

In [3]:
for line in inp_train:
    previous = "<s>"
    context[previous] += 1
    wordtags = line.split()
    for wordtag in wordtags:
        word,tag = wordtag.split("_")
        transit[previous+" "+tag] +=1
        context[tag] += 1
        emit[tag+" "+word] +=1
        previous = tag
    transit[previous+" </s>"] += 1

transition = defaultdict(lambda:0)
emission = defaultdict(lambda:0)
possible_tags = context
for key,value in transit.items():
    previous,word = key.split()
    transition[key] = float(value)/context[previous]
for key,value in emit.items():
    tag,word = key.split()
    emission[key] = float(value)/context[tag]

In [4]:
ans = ""
lambda_E = 0.95
N = 1000000
for line in inp:
    words = line.split()
    l = len(words)
    best_score = defaultdict(lambda: N)
    best_edge = defaultdict(lambda: None)
    best_score["0 <s>"] = 0
    best_edge["0 <s>"] = None
    for i in range(0,l):
        for prev in possible_tags.keys():
            for after in possible_tags.keys():
                if (str(i)+" "+prev)in best_score and (prev+" "+after)in transition:
                    score_T = transition[prev+" "+after]
                    score_E = lambda_E*emission[after+" "+words[i]] + (1-lambda_E)/N
                    score = best_score[str(i)+" "+prev]-math.log(score_T,2)-math.log(score_E,2)
                    if (str(i+1)+" "+after) not in best_score or best_score[str(i+1)+" "+after] > score:
                        best_score[str(i+1)+" "+after] = score
                        best_edge[str(i+1)+" "+after] = str(i)+" "+prev

    for tag in possible_tags.keys():
        if (str(l)+" "+tag)in best_score and (tag+" </s>")in transition:
            score_T = transition[tag+" </s>"]
            score = best_score[str(l)+" "+tag]-math.log(score_T,2)
            if (str(l+1)+" </s>") not in best_score or best_score[str(l+1)+" </s>"] > score:
                best_score[str(l+1)+" </s>"] = score
                best_edge[str(l+1)+" </s>"] = str(l)+" "+tag
    
    tags = []
    next_edge = best_edge[str(l+1)+" </s>"]
    while next_edge != "0 <s>":
        position,tag = next_edge.split()
        tags.append(tag)
        next_edge = best_edge[next_edge]
    tags.reverse()
    ans += " ".join(tags)
    ans += "\n"
with open("../data/my_answer.pos","w") as wf:
    wf.write(ans)

ValueError: math domain error